In [1]:
import numpy as np
from numpy import linalg as la

In [2]:
A = np.array([[0, 4, 6, 0], [1, 0, 3, 4], [2, 0, 3, 0]])
A

array([[0, 4, 6, 0],
       [1, 0, 3, 4],
       [2, 0, 3, 0]])

numpy的SVD部分分解

In [3]:
U, S, V_t = la.svd(A, 0)

In [4]:
U

array([[ 0.82865413,  0.49969725, -0.25225978],
       [ 0.43576771, -0.85873563, -0.26959158],
       [ 0.35133863, -0.11347151,  0.92934675]])

In [5]:
S

array([8.31249063, 4.12059922, 2.21881987])

In [6]:
V_t

array([[ 0.13695594,  0.39875131,  0.88219574,  0.20969297],
       [-0.26347591,  0.48507241,  0.01979374, -0.83360267],
       [ 0.7161924 , -0.45476388,  0.20988942, -0.48600895]])

结果验证

In [7]:
I = np.eye(3,3)
S2 = I * S
np.dot(np.dot(U, S2), V_t)

array([[ 4.48143437e-16,  4.00000000e+00,  6.00000000e+00,
        -8.02283588e-16],
       [ 1.00000000e+00,  3.81414949e-16,  3.00000000e+00,
         4.00000000e+00],
       [ 2.00000000e+00, -1.02784819e-16,  3.00000000e+00,
         5.48675884e-16]])

降维结果验证

In [8]:
S3 = S2[0:2, 0:2]
U2 = U[:,0:2]
V_t2 = V_t[0:2,:]
np.dot(np.dot(U2, S3), V_t2)

array([[ 0.40086651,  3.74546001,  6.1174791 , -0.27202845],
       [ 1.4284085 , -0.27202845,  3.12555063,  3.70928152],
       [ 0.52317329,  0.93774724,  2.56719689,  1.00217623]])

## 步骤1 计算A的转置和A<sup>T</sup>A

In [9]:
A.transpose()

array([[0, 1, 2],
       [4, 0, 0],
       [6, 3, 3],
       [0, 4, 0]])

In [10]:
AtA = np.dot(A.transpose(), A)  #X的转置与X的矩阵乘积
AtA

array([[ 5,  0,  9,  4],
       [ 0, 16, 24,  0],
       [ 9, 24, 54, 12],
       [ 4,  0, 12, 16]])

## 步骤2 确定A<sup>T</sup>A的特征值

In [11]:
lam, vector = la.eig(AtA)  #A的转置与A的矩阵乘积的特征值和特征向量

In [12]:
lam

array([6.90975004e+01, 1.69793380e+01, 2.47426309e-15, 4.92316164e+00])

In [13]:
vector

array([[ 0.13695594,  0.26347591,  0.63157895, -0.7161924 ],
       [ 0.39875131, -0.48507241,  0.63157895,  0.45476388],
       [ 0.88219574, -0.01979374, -0.42105263, -0.20988942],
       [ 0.20969297,  0.83360267,  0.15789474,  0.48600895]])

验证特征值

In [14]:
for i in range(4):
    I = np.eye(4, 4, dtype=float)
    X = I * lam[i]
    print(la.det(AtA-X))  #验证行列式是否为0

6.091582268188108e-09
2.0826999696654314e-11
-1.5944578990456706e-11
4.162547661234323e-12


取最大三位特征值

In [15]:
lam = lam[[0, 1, 3]]
lam

array([69.0975004 , 16.97933796,  4.92316164])

## 步骤3 根据特征值构建对角矩阵S和S<sup>-1</sup>

特征值开方

In [16]:
s = np.sqrt(lam)
s

array([8.31249063, 4.12059922, 2.21881987])

构造S

In [17]:
I = np.eye(3, 3, dtype=float)
S = I * s
S

array([[8.31249063, 0.        , 0.        ],
       [0.        , 4.12059922, 0.        ],
       [0.        , 0.        , 2.21881987]])

构造S<sup>-1</sup>

In [18]:
S_inv = la.inv(S)
S_inv

array([[0.12030089, 0.        , 0.        ],
       [0.        , 0.24268315, 0.        ],
       [0.        , 0.        , 0.45069003]])

## 步骤4 根据A<sup>T</sup>A特征向量生成V和V<sup>t</sup>

根据特征向量X1,X2,X3构造V=(X1,X2,X3)

由于特征向量无唯一解，取已得到的SVD分解的V做特征向量，代入进行验证

In [19]:
for i in range(3):
    I = np.eye(4, 4, dtype=float)
    X = I * lam[i]
    print("i =",i)
    print(AtA - X)
    print(V_t[i])
    print(np.dot(AtA - X, V_t[i]))  #检验乘积结果是否为0向量
    print("\n")

i = 0
[[-64.0975004   0.          9.          4.       ]
 [  0.        -53.0975004  24.          0.       ]
 [  9.         24.        -15.0975004  12.       ]
 [  4.          0.         12.        -53.0975004]]
[0.13695594 0.39875131 0.88219574 0.20969297]
[-1.24344979e-14 -7.10542736e-15 -2.13162821e-14 -7.54951657e-15]


i = 1
[[-11.97933796   0.           9.           4.        ]
 [  0.          -0.97933796  24.           0.        ]
 [  9.          24.          37.02066204  12.        ]
 [  4.           0.          12.          -0.97933796]]
[-0.26347591  0.48507241  0.01979374 -0.83360267]
[-8.88178420e-16  1.16573418e-15 -1.77635684e-15 -1.33226763e-15]


i = 2
[[ 0.07683836  0.          9.          4.        ]
 [ 0.         11.07683836 24.          0.        ]
 [ 9.         24.         49.07683836 12.        ]
 [ 4.          0.         12.         11.07683836]]
[ 0.7161924  -0.45476388  0.20988942 -0.48600895]
[2.35228503e-15 8.88178420e-16 7.99360578e-15 6.66133815e-15]




## 步骤5 计算U

U = AVS<sup>-1<sup>

In [20]:
A

array([[0, 4, 6, 0],
       [1, 0, 3, 4],
       [2, 0, 3, 0]])

In [21]:
V = V_t.transpose()
V

array([[ 0.13695594, -0.26347591,  0.7161924 ],
       [ 0.39875131,  0.48507241, -0.45476388],
       [ 0.88219574,  0.01979374,  0.20988942],
       [ 0.20969297, -0.83360267, -0.48600895]])

In [22]:
S_inv

array([[0.12030089, 0.        , 0.        ],
       [0.        , 0.24268315, 0.        ],
       [0.        , 0.        , 0.45069003]])

In [23]:
temp = np.dot(V, S_inv)
temp

array([[ 0.01647592, -0.06394116,  0.32278077],
       [ 0.04797014,  0.1177189 , -0.20495755],
       [ 0.10612893,  0.00480361,  0.09459507],
       [ 0.02522625, -0.20230132, -0.21903939]])

In [24]:
np.dot(A, temp)          

array([[ 0.82865413,  0.49969725, -0.25225978],
       [ 0.43576771, -0.85873563, -0.26959158],
       [ 0.35133863, -0.11347151,  0.92934675]])

同直接得到的U参考比较

In [25]:
U

array([[ 0.82865413,  0.49969725, -0.25225978],
       [ 0.43576771, -0.85873563, -0.26959158],
       [ 0.35133863, -0.11347151,  0.92934675]])